# Run Agent
- Implement a workflow to write a daily AI newsletter
- see README.md for details


In [1]:
import os
import yaml
import dotenv
import logging
import json
import yaml
from datetime import datetime
import time
import random
import glob
import pickle
import sqlite3

from pathlib import Path

import asyncio
import nest_asyncio

import pydantic
from pydantic import BaseModel, Field, RootModel
from typing import Dict, TypedDict, Type, List, Optional, Any, Iterable, Text
from dataclasses import dataclass, field
from enum import Enum

import numpy as np
import pandas as pd

import pandas as pd
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
import hdbscan

import openai
from openai import AsyncOpenAI

import agents
from agents.exceptions import InputGuardrailTripwireTriggered
from agents import (Agent, Runner, Tool, OpenAIResponsesModel, 
                    ModelSettings, FunctionTool, InputGuardrail, GuardrailFunctionOutput,
                    SQLiteSession, set_default_openai_api, set_default_openai_client
                   )


import tenacity
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type

from IPython.display import HTML, Image, Markdown, display

from log_handler import SQLiteLogHandler, setup_sqlite_logging, sanitize_error_for_logging
from config import LOGDB
from llm import LLMagent, LangfuseClient  # methods to apply prompts async to large batches
from db import Url 

from fetch import Fetcher # fetch news urls
from newsletter_state import NewsletterAgentState, StepStatus
from news_agent import NewsletterAgent


In [2]:
print(f"OpenAI:            {openai.__version__}")
print(f"OpenAI Agents SDK  {agents.__version__}")
print(f"Pydantic           {pydantic.__version__}")


OpenAI:            1.109.0
OpenAI Agents SDK  0.3.1
Pydantic           2.11.9


In [3]:
dotenv.load_dotenv()

# to run async in jupyter notebook
nest_asyncio.apply()

# verbose OpenAI console logging if something doesn't work
# logging.basicConfig(level=logging.DEBUG)
# openai_logger = logging.getLogger("openai")
# openai_logger.setLevel(logging.DEBUG)


In [4]:
# modules create a default logger, or we can pass this logger

def setup_logging(session_id: str = "default", db_path: str = "agent_logs.db") -> logging.Logger:
    """Set up logging to console and SQLite database."""

    # Create logger
    logging.basicConfig(level=logging.INFO)

    logger = logging.getLogger(f"NewsletterAgent.{session_id}")
    logger.setLevel(logging.INFO)

    # Clear any existing handlers
    logger.handlers.clear()

    # Console handler
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)
    console_formatter = logging.Formatter(
        '%(asctime)s | %(name)s | %(levelname)s | %(message)s',
        datefmt='%H:%M:%S'
    )
    console_handler.setFormatter(console_formatter)

    # SQLite handler
    sqlite_handler = SQLiteLogHandler(db_path)
    sqlite_handler.setLevel(logging.INFO)
    sqlite_formatter = logging.Formatter('%(message)s')
    sqlite_handler.setFormatter(sqlite_formatter)

    # Add handlers to logger
    logger.addHandler(console_handler)
    logger.addHandler(sqlite_handler)

    # Prevent propagation to root logger
    logger.propagate = False

    return logger

logger = setup_logging("newsletter_agent", "test_logs.db")

# Log some test messages
logger.info("Test info message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.warning("Test warning message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.error("Test error message", extra={
    'step_name': 'error_step',
    'agent_session': 'demo_session'
})

sanitize_error_for_logging("log with some bad stuff for the filter: sk-proj-123456789012345678901234567890123456789012345678")

09:47:32 | NewsletterAgent.newsletter_agent | INFO | Test info message
09:47:32 | NewsletterAgent.newsletter_agent | WARNING | Test warning message
09:47:32 | NewsletterAgent.newsletter_agent | ERROR | Test error message


'log with some bad stuff for the filter: [API_KEY_REDACTED]'

# Run Agent Worfklow

In [5]:
print("🚀 Creating NewsletterAgent...")

do_download=False
process_since=None
# process_since='2025-10-05 18:30:00'

try:
    # set up state
    session_id = 'test_newsletter_20251014091057369527'
    step_name = 'step_03_download_articles'
#     del session_id, step_name
except Exception as e:
    print(e)

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY environment variable not set")

# Set up OpenAI client for the agents SDK
set_default_openai_client(AsyncOpenAI(api_key=api_key))

# Create agent with persistent state
if 'session_id' in vars():
    # load state from db for session_id and state
    print("session_id is defined")
    print(session_id)
    state = NewsletterAgentState(session_id=session_id, 
                                 db_path="newsletter_agent.db", 
                                 do_download=do_download,
                                 process_since=process_since,
                                 verbose=False
                                )
    state = state.load_from_db(step_name)
    agent = NewsletterAgent(session_id=session_id, state=state, verbose=True, timeout=30)    
else:
    # create new session
    print("session_id is not defined")
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S%f")    
    session_id = f"test_newsletter_{timestamp}"
    print(session_id)
    state = NewsletterAgentState(session_id=session_id, 
                                 db_path="newsletter_agent.db",
                                 do_download=do_download,
                                 process_since=process_since,
                                 verbose=False
                                ) 
    agent = NewsletterAgent(session_id=session_id, state=state, verbose=False, timeout=30)
    state.serialize_to_db("initialize")

09:47:34 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Using provided state with 92 articles


🚀 Creating NewsletterAgent...
session_id is defined
test_newsletter_20251014091057369527
Initialized NewsletterAgent with persistent state and 9-step workflow
Session ID: test_newsletter_20251014091057369527


In [6]:
agent.state.get_status()


{'headlines': {'total': 0},
 'sources': {'config_file': 'sources.yaml', 'loaded_sources': 0},
 'topics': {'cluster_topics': 0, 'topics': []},
 'workflow': {'current_step': 'step_01_fetch_urls',
  'workflow_complete': False,
  'workflow_status': 'not_started',
  'workflow_status_message': '',
  'progress_percentage': 0.0,
  'max_edits': 2,
  'concurrency': 16,
  'do_download': True,
  'process_since': None},
 'processing': {'topic_clusters': 0,
  'newsletter_sections': 0,
  'final_newsletter_length': 0}}

In [7]:
# User prompt to run workflow
user_prompt = "Show the workflow status"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


📝 User prompt: 'Show the workflow status'


09:11:16 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Starting check_workflow_status
09:11:16 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Completed check_workflow_status


⏱️  Total execution time: 13.47s
📊 Final result:
Current workflow status:

- Progress: 0.0% (0/9 complete)
- Summary: 0 complete, 0 started, 0 failed, 9 not started
- Next step: Step 1 — Gather URLs (Fetch Urls)

Step-by-step status:
- Step 1: Fetch Urls — not_started
- Step 2: Filter Urls — not_started
- Step 3: Download Articles — not_started
- Step 4: Extract Summaries — not_started
- Step 5: Rate Articles — not_started
- Step 6: Cluster By Topic — not_started
- Step 7: Select Sections — not_started
- Step 8: Draft Sections — not_started
- Step 9: Finalize Newsletter — not_started

Would you like me to start Step 1 (gather URLs) now, run all remaining steps to create the newsletter, or run a specific step?


In [8]:
# User prompt to run a workflow step
user_prompt = "Run step 1, fetch urls"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)



📝 User prompt: 'Run step 1, fetch urls'


09:11:35 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Starting check_workflow_status
09:11:35 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Completed check_workflow_status
09:11:40 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Starting Step 1: Gather URLs
09:11:40 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Cleaning download/sources: 
09:11:40 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Successfully cleaned download/sources
09:11:40 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Cleaning download/text: 
09:11:40 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Successfully cleaned download/text
09:11:40 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Cleaning download/html: 
09:11:40 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Successfully cleaned download/html
2025-10-14 09:11:40,641 - fetcher_5034606160 - INFO - [fetcher_in

▶ Starting Step 1: step_01_fetch_urls


2025-10-14 09:11:40,817 - fetcher_5034606160 - INFO - [fetch_rss] RSS fetch successful for New York Times: 20 articles
2025-10-14 09:11:40,818 - fetcher_5034606160 - INFO - [fetch_html] Fetching HTML from Reddit: https://www.reddit.com/r/AI_Agents+ArtificialInteligence+Automate+ChatGPT+ChatGPTCoding+Futurology+MachineLearning+OpenAI+ProgrammerHumor+accelerate+aiArt+aivideo+artificial+deeplearning+learnmachinelearning+programming+singularity+tech+technews+technology/top/?sort=top&t=day
2025-10-14 09:11:40,895 - fetcher_5034606160 - INFO - [fetch_rss] RSS fetch successful for HackerNoon: 50 articles
2025-10-14 09:11:40,896 - fetcher_5034606160 - INFO - [fetch_html] Fetching HTML from Techmeme: https://www.techmeme.com/river
2025-10-14 09:11:41,050 - fetcher_5034606160 - INFO - [fetch_rss] RSS fetch successful for Ars Technica: 20 articles
2025-10-14 09:11:41,106 - fetcher_5034606160 - INFO - [fetch_rss] Fetching RSS from The Register: https://www.theregister.com/software/ai_ml/headlines.

2025-10-14 09:11:48,046 - fetcher_5034606160 - INFO - Starting scrape_source https://www.bloomberg.com/ai, Bloomberg
2025-10-14 09:11:48,047 - fetcher_5034606160 - INFO - scrape_url(https://www.bloomberg.com/ai)
2025-10-14 09:11:48,048 - fetcher_5034606160 - INFO - scraping https://www.bloomberg.com/ai to download/sources
2025-10-14 09:11:48,048 - fetcher_5034606160 - INFO - Downloading https://www.bloomberg.com/ai
2025-10-14 09:11:48,050 - fetcher_5034606160 - INFO - [fetch_html] Source dict for Reddit: {'type': 'html', 'url': 'https://www.reddit.com/r/AI_Agents+ArtificialInteligence+Automate+ChatGPT+ChatGPTCoding+Futurology+MachineLearning+OpenAI+ProgrammerHumor+accelerate+aiArt+aivideo+artificial+deeplearning+learnmachinelearning+programming+singularity+tech+technews+technology/top/?sort=top&t=day', 'filename': 'Reddit', 'exclude': ['^https://www.reddit.com/user/', '^https://www.reddit.com/domain/', '^https://chat.reddit.com/', '^https://i.redd.it/', '^https://redditblog.com/', '^ht

...

2025-10-14 09:12:13,980 - fetcher_5034606160 - DEBUG - Attempting to parse last_updated: '10/15/2025 00:12:13' (type: <class 'str'>)
2025-10-14 09:12:13,981 - fetcher_5034606160 - DEBUG - Parsed datetime: 2025-10-15 00:12:13
2025-10-14 09:12:13,982 - fetcher_5034606160 - DEBUG - Added UTC timezone: 2025-10-15 00:12:13+00:00
2025-10-14 09:12:13,982 - fetcher_5034606160 - DEBUG - Converted to UTC: 2025-10-15 00:12:13+00:00
2025-10-14 09:12:13,982 - fetcher_5034606160 - DEBUG - Formatted last_updated: 2025-10-15T00:12:13Z
2025-10-14 09:12:13,983 - fetcher_5034606160 - INFO - Saving HTML to download/sources/WSJ.html
2025-10-14 09:12:13,994 - fetcher_5034606160 - INFO - [fetch_html] Parsing HTML file: download/sources/WSJ.html
2025-10-14 09:12:14,030 - fetcher_5034606160 - INFO - [fetch_html] Parsed HTML file: download/sources/WSJ.html
2025-10-14 09:12:14,031 - fetcher_5034606160 - INFO - [fetch_html] HTML fetch successful for WSJ: 26 articles
2025-10-14 09:12:15,644 - fetcher_5034606160 - 

,source,url
0,Ars Technica,20
1,Bloomberg,28
2,Business Insider,16
3,FT,46
4,Feedly AI,65
5,Hacker News,29
6,HackerNoon,50
7,New York Times,19
8,NewsAPI,97
9,Reddit,46


,source,title,url,published,rss_summary,id
0,Ars Technica,Google’s Photoshop-killer AI model is coming t...,https://arstechnica.com/google/2025/10/googles...,"Mon, 13 Oct 2025 19:52:23 +0000","After more than 5 billion AI image edits, Nano...",0
1,Ars Technica,"To shield kids, California hikes fake nude fin...",https://arstechnica.com/tech-policy/2025/10/to...,"Mon, 13 Oct 2025 18:52:06 +0000",California cracks down on AI as child safety c...,1
2,Ars Technica,OpenAI will stop saving most ChatGPT users’ de...,https://arstechnica.com/tech-policy/2025/10/op...,"Fri, 10 Oct 2025 14:58:29 +0000",Court ends controversial order forcing OpenAI ...,2
3,Ars Technica,“Like putting on glasses for the first time”—h...,https://arstechnica.com/science/2025/10/like-p...,"Fri, 10 Oct 2025 11:00:24 +0000",AI is “comically good” at detecting small eart...,3
4,Ars Technica,AI models can acquire backdoors from surprisin...,https://arstechnica.com/ai/2025/10/ai-models-c...,"Thu, 09 Oct 2025 22:03:21 +0000","Anthropic study suggests ""poison"" training att...",4
...,...,...,...,...,...,...
719,NewsAPI,USA Business Intelligence Market Outlook Repor...,https://www.globenewswire.com/news-release/202...,2025-10-13T10:10:00Z,NaN,719
720,NewsAPI,Brazil’s Invisible Communities Deserve to Be C...,http://foreignpolicy.com/2025/10/13/brazil-qui...,2025-10-13T11:00:51Z,NaN,720
721,NewsAPI,Far fewer Americans support political violence...,https://theconversation.com/far-fewer-american...,2025-10-13T12:38:15Z,NaN,721
722,NewsAPI,Saudi Arabia Automated Storage And Retrieval S...,https://www.globenewswire.com/news-release/202...,2025-10-13T11:29:00Z,NaN,722


09:12:24 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Completed Step 1: Gathered 845 articles
09:12:26 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Starting check_workflow_status
09:12:26 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Completed check_workflow_status


⏱️  Total execution time: 63.46s
📊 Final result:
Step 1 (Fetch URLs) completed.

Summary:
- Collected 845 headlines from 17 RSS sources; 724 article records stored in persistent state.
- Progress: 11.1% (1/9 complete)
- Next step: Step 2 — Filter URLs (AI-related only)

Would you like me to continue and run Step 2 now?


In [9]:
pd.DataFrame(state.headline_data) 


,source,title,url,published,rss_summary,id
0,Ars Technica,Google’s Photoshop-killer AI model is coming t...,https://arstechnica.com/google/2025/10/googles...,"Mon, 13 Oct 2025 19:52:23 +0000","After more than 5 billion AI image edits, Nano...",0
1,Ars Technica,"To shield kids, California hikes fake nude fin...",https://arstechnica.com/tech-policy/2025/10/to...,"Mon, 13 Oct 2025 18:52:06 +0000",California cracks down on AI as child safety c...,1
2,Ars Technica,OpenAI will stop saving most ChatGPT users’ de...,https://arstechnica.com/tech-policy/2025/10/op...,"Fri, 10 Oct 2025 14:58:29 +0000",Court ends controversial order forcing OpenAI ...,2
3,Ars Technica,“Like putting on glasses for the first time”—h...,https://arstechnica.com/science/2025/10/like-p...,"Fri, 10 Oct 2025 11:00:24 +0000",AI is “comically good” at detecting small eart...,3
4,Ars Technica,AI models can acquire backdoors from surprisin...,https://arstechnica.com/ai/2025/10/ai-models-c...,"Thu, 09 Oct 2025 22:03:21 +0000","Anthropic study suggests ""poison"" training att...",4
...,...,...,...,...,...,...
719,NewsAPI,USA Business Intelligence Market Outlook Repor...,https://www.globenewswire.com/news-release/202...,2025-10-13T10:10:00Z,NaN,719
720,NewsAPI,Brazil’s Invisible Communities Deserve to Be C...,http://foreignpolicy.com/2025/10/13/brazil-qui...,2025-10-13T11:00:51Z,NaN,720
721,NewsAPI,Far fewer Americans support political violence...,https://theconversation.com/far-fewer-american...,2025-10-13T12:38:15Z,NaN,721
722,NewsAPI,Saudi Arabia Automated Storage And Retrieval S...,https://www.globenewswire.com/news-release/202...,2025-10-13T11:29:00Z,NaN,722


In [10]:
countdf = pd.DataFrame(state.headline_data) \
    .groupby("source") \
    .count()[["id"]] \
    .reset_index() \
    .rename(columns={'id': 'count'}) \
    .sort_values("count", ascending=False)
countdf 


,source,count
10,Techmeme,160
8,NewsAPI,97
4,Feedly AI,65
6,HackerNoon,50
12,The Register,50
3,FT,46
9,Reddit,46
13,The Verge,31
5,Hacker News,29
1,Bloomberg,28


In [11]:
# Run tool directly without LLM processing an input prompt or results
# user_prompt = "Run step 2, filter urls"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_tool_direct("filter_urls")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


09:12:35 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Starting Step 2: Filter URLs
09:12:35 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | 🔍 Filtering 724 headlines...
09:12:35 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | 🔄 Checking for duplicates (all urls without date restrictions)
09:12:35 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | 🔍 Filtering 724 articles for dupes.
09:12:35 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | URL deduplication: 469 duplicates filtered, 255 new URLs remain
09:12:35 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | 🔍 Filtering 724 headlines for AI relevance using LLM...


▶ Starting Step 2: step_02_filter_urls
checking https://arstechnica.com/google/2025/10/googles-nano-banana-ai-image-editor-is-coming-to-search-photos-and-notebooklm/
found url, no cutoff set - treating as duplicate
checking https://arstechnica.com/tech-policy/2025/10/to-shield-kids-california-hikes-fake-nude-fines-to-250k-max/
found url, no cutoff set - treating as duplicate
checking https://arstechnica.com/tech-policy/2025/10/openai-no-longer-forced-to-save-deleted-chats-but-some-users-still-affected/
found url, no cutoff set - treating as duplicate
checking https://arstechnica.com/science/2025/10/like-putting-on-glasses-for-the-first-time-how-ai-improves-earthquake-detection/
found url, no cutoff set - treating as duplicate
checking https://arstechnica.com/ai/2025/10/ai-models-can-acquire-backdoors-from-surprisingly-few-malicious-documents/
found url, no cutoff set - treating as duplicate
checking https://arstechnica.com/ai/2025/10/bank-of-england-warns-ai-stock-bubble-rivals-2000-do

INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/filter_urls' from Langfuse
INFO:llm:Parsed prompt 'newsagent/filter_urls': model=gpt-5-nano, system_len=459, user_len=954
09:12:36 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Processing 11 chunks with concurrency 16


concurrency:  16


09:13:12 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Completed Step 2: 93 AI-related articles, 469 duplicates removed


⏱️  Total execution time: 36.77s
📊 Final result:
✅ Step 2 step_02_filter_urls completed successfully! Removed 469 duplicate URLs, classified 255 new articles, found 93 AI-related.


In [12]:
# User prompt to run workflow
# user_prompt = "Run step 3, download full articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_tool_direct("download_articles")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

09:13:12 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Starting Step 3: Download Articles
09:13:12 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Starting concurrent scraping of 93 AI-related articles


▶ Starting Step 3: step_03_download_articles


09:13:12 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Launching browser for 93 URLs with 16 concurrent workers
09:13:12 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Worker 0 fetching 1 of 93 https://finance.yahoo.com/news/openai-taps-broadcom-build-first-130111782.html
09:13:12 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | scrape_url(https://finance.yahoo.com/news/openai-taps-broadcom-build-first-130111782.html)
09:13:12 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | scraping https://finance.yahoo.com/news/openai-taps-broadcom-build-first-130111782.html to download/html
09:13:12 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Downloading https://finance.yahoo.com/news/openai-taps-broadcom-build-first-130111782.html
09:13:12 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Worker 1 fetching 2 of 93 https://finance.yahoo.com/video/ai-stocks-bubble-bofa-fund-110412556.html
09:13:12 | Ne

09:13:12 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | scrape_url(https://fortune.com/2025/10/13/workslop-wasting-workers-time-ai-resentment/)
09:13:12 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | scraping https://fortune.com/2025/10/13/workslop-wasting-workers-time-ai-resentment/ to download/html
09:13:12 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Downloading https://fortune.com/2025/10/13/workslop-wasting-workers-time-ai-resentment/
09:13:12 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Worker 11 fetching 11 of 93 https://www.bloomberg.com/news/articles/2025-10-14/europe-aims-for-ai-independence-to-avoid-reliance-on-us-and-china
09:13:12 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Skipping ignored domain: www.bloomberg.com
09:13:12 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Worker 11 completed https://www.bloomberg.com/news/articles/2025-10-14/europe-aims-for-ai-indepe

09:13:14 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Downloading https://techcrunch.com/2025/10/14/google-to-invest-15b-in-indian-ai-infrastructure-hub/
09:13:14 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Worker 13 fetching 19 of 93 https://biztoc.com/x/9b44eaa976db33c5
09:13:15 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Rate limiting domain biztoc.com, will retry later (need to wait 1.9s)
09:13:15 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Worker 13 re-queued rate-limited URL: https://biztoc.com/x/9b44eaa976db33c5
09:13:15 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Response: 200
09:13:15 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Response: 200
09:13:15 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Response: 200
09:13:15 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Response: 200
09:13:15 | NewsletterAgent.test_newsletter_20251

...

concurrency:  16


09:26:30 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Inserted 2 new site records
09:26:30 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Updated 93 URL records with final URLs


Starting with 93 rows...
Processing 93 files...
Reading and truncating files to 8192 tokens using text-embedding-3-large tokenizer...
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Getting embeddings for 80 texts...


09:26:35 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Completed Step 3: Downloaded 80 articles


Creating indexed similarity matrix...
Finding pairs with similarity > 0.925...
Filtering dataframe...
  Pair: 49 vs 27
    49: The Register - Shadow AI: Staffers are bringing AI tools they use at home to work, warns Microsoft
    27: Feedly AI - Shadow AI: Staffers are bringing AI tools they use at home to work, warns Microsoft
Removing 1 rows due to high similarity 
Final dataframe has 92 rows (removed 1 rows)
⏱️  Total execution time: 802.85s
📊 Final result:
✅ Step 3 step_03_download_articles completed successfully! Downloaded 80 AI-related articles with 87% success rate.
📊 Average article length: 3909 characters
🔗 Content stored in persistent state.


In [13]:
headline_df = state.headline_df
with pd.option_context('display.max_columns', None, 'display.width', None, 'display.max_colwidth', None  ):
    display(headline_df[['id', 'source', 'title', 'status', 'final_url']].loc[headline_df["status"]!=200])




,id,source,title,status,final_url
79,28,Bloomberg,Europe Fights for AI Independence to Avoid Becoming Tech ‘Colony’,skipped,https://www.bloomberg.com/news/articles/2025-10-14/europe-aims-for-ai-independence-to-avoid-reliance-on-us-and-china
80,24,Bloomberg,Pony AI and WeRide Get Chinese Regulator’s Nod to List in Hong Kong,skipped,https://www.bloomberg.com/news/articles/2025-10-14/pony-ai-weride-get-chinese-regulator-s-nod-to-list-in-hong-kong
81,25,Bloomberg,"AI Stocks Are in a Bubble, Most Investors Say in BofA Survey",skipped,https://www.bloomberg.com/news/articles/2025-10-14/most-investors-say-ai-stocks-are-in-a-bubble-bofa-poll-shows
82,27,Bloomberg,"UK Lacks Growth Funding for Critical Sectors Like Quantum, AI",skipped,https://www.bloomberg.com/news/articles/2025-10-14/uk-lacks-growth-funding-for-critical-sectors-like-quantum-ai
83,23,Bloomberg,"Tonal Eyes Growth with Retail Expansion, AI-Powered Pilates",skipped,https://www.bloomberg.com/news/articles/2025-10-14/tonal-adds-reformer-pilates-as-it-expands-retail-presence-returns-to-growth
84,26,Bloomberg,Publicis Raises Growth Outlook on Strong Ads and AI Demand,skipped,https://www.bloomberg.com/news/articles/2025-10-14/publicis-raises-growth-outlook-on-strong-ads-and-ai-demand
85,21,Bloomberg,AMD Says Oracle Is Committing to Widespread Use of New AI Chips,skipped,https://www.bloomberg.com/news/articles/2025-10-14/amd-says-oracle-is-committing-to-widespread-use-of-new-ai-chips
86,122,Feedly AI,Leak Says Google Might Release The Gemini 3.0 Model On October 22,404,https://www.bgr.com/1996171/google-gemini-3-release-october-22/
87,329,Techmeme,OpenAI's massive deals show Sam Altman is selling a vision of a world-changing product and achieving it via world-changing financial engineering to raise $1T+,skipped,https://www.bloomberg.com/opinion/newsletters/2025-10-13/openai-keeps-doing-deals
88,577,WSJ,"Publicis Lifts Outlook, Says AI Is Boosting Growth",skipped,https://www.wsj.com/tech/ai/publicis-lifts-outlook-says-ai-is-boosting-growth-616a3546


In [14]:
headline_df = state.headline_df
with pd.option_context('display.max_columns', None, 'display.width', None, 'display.max_colwidth', None  ):
    display(headline_df[['id', 'source', 'title', 'status', 'final_url']].loc[headline_df["html_path"]==""])



,id,source,title,status,final_url
79,28,Bloomberg,Europe Fights for AI Independence to Avoid Becoming Tech ‘Colony’,skipped,https://www.bloomberg.com/news/articles/2025-10-14/europe-aims-for-ai-independence-to-avoid-reliance-on-us-and-china
80,24,Bloomberg,Pony AI and WeRide Get Chinese Regulator’s Nod to List in Hong Kong,skipped,https://www.bloomberg.com/news/articles/2025-10-14/pony-ai-weride-get-chinese-regulator-s-nod-to-list-in-hong-kong
81,25,Bloomberg,"AI Stocks Are in a Bubble, Most Investors Say in BofA Survey",skipped,https://www.bloomberg.com/news/articles/2025-10-14/most-investors-say-ai-stocks-are-in-a-bubble-bofa-poll-shows
82,27,Bloomberg,"UK Lacks Growth Funding for Critical Sectors Like Quantum, AI",skipped,https://www.bloomberg.com/news/articles/2025-10-14/uk-lacks-growth-funding-for-critical-sectors-like-quantum-ai
83,23,Bloomberg,"Tonal Eyes Growth with Retail Expansion, AI-Powered Pilates",skipped,https://www.bloomberg.com/news/articles/2025-10-14/tonal-adds-reformer-pilates-as-it-expands-retail-presence-returns-to-growth
84,26,Bloomberg,Publicis Raises Growth Outlook on Strong Ads and AI Demand,skipped,https://www.bloomberg.com/news/articles/2025-10-14/publicis-raises-growth-outlook-on-strong-ads-and-ai-demand
85,21,Bloomberg,AMD Says Oracle Is Committing to Widespread Use of New AI Chips,skipped,https://www.bloomberg.com/news/articles/2025-10-14/amd-says-oracle-is-committing-to-widespread-use-of-new-ai-chips
86,122,Feedly AI,Leak Says Google Might Release The Gemini 3.0 Model On October 22,404,https://www.bgr.com/1996171/google-gemini-3-release-october-22/
87,329,Techmeme,OpenAI's massive deals show Sam Altman is selling a vision of a world-changing product and achieving it via world-changing financial engineering to raise $1T+,skipped,https://www.bloomberg.com/opinion/newsletters/2025-10-13/openai-keeps-doing-deals
88,577,WSJ,"Publicis Lifts Outlook, Says AI Is Boosting Growth",skipped,https://www.wsj.com/tech/ai/publicis-lifts-outlook-says-ai-is-boosting-growth-616a3546


In [21]:
agent.verbose=True


In [6]:
# User prompt to run workflow
# user_prompt = "Run step 4, Summarize articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("extract_summaries")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

09:47:48 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Processing 92 AI articles for summarization
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/extract_summaries' from Langfuse
INFO:llm:Parsed prompt 'newsagent/extract_summaries': model=gpt-5-mini, system_len=1273, user_len=43
09:47:48 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Using model 'gpt-5-mini' for summarization
09:47:48 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Initialized LLMagent:
system_prompt: You are an expert AI news analyst. Your task is to create concise, informative bullet-point summaries of AI and technology articles for a professional newsletter audience.

You will receive a list of JSON object with fields "id" and "title"
Return **only** a JSON object that satisfies the provided schema.
For each article provided, you MUST return one element with the same id, and the summary.
Return elements in the same order they w

▶ Starting Step 4: step_04_extract_summaries
concurrency:  16


09:47:48 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | User message: Summarize the article below:

[
  {
    "id":717,
    "text_content":"Page title: AI Didn\u2019t Steal Our Intelligence \u2014 We Abandoned It!\nSocial card title: AI Didn\u2019t Steal Our Intelligence \u2014 We Abandoned It!\nSocial card description: AI\u2019s incomparable breadth of knowledge, pattern recognition and rhetorical control should inspire us to refine our own intelligence.\nEver since the European Enlightenment that ushered in what we think of as \u201cthe age of science\u201d some three centuries ago, the human race \u2014 or at least its social elites \u2014 have been entertaining the belief that we possess a cosmic monopoly on a commodity known as reasoning. Is that trend now being interrupted by the emergence of a rival also capable of reasoning and that might even surpass us? Is AI the Godzilla we need to defeat or tame?\nBefore the scientific age, there were rivals to human reason

09:47:48 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | User message: Summarize the article below:

[
  {
    "id":560,
    "text_content":"Page title: Visa just launched a protocol to secure the AI shopping boom \u2014 here\u2019s what it means for merchants | VentureBeat\nSocial card title: Visa just launched a protocol to secure the AI shopping boom \u2014 here\u2019s what it means for merchants\nSocial card description: Visa launches a new Trusted Agent Protocol to verify AI shopping bots, aiming to secure online commerce as agentic shopping traffic surges 4,700%.\nVisa is introducing a new security framework designed to solve one of the thorniest problems emerging in artificial intelligence-powered commerce: how retailers can tell the difference between legitimate AI shopping assistants and the malicious bots that plague their websites.\nThe payments giant unveiled its Trusted Agent Protocol on Tuesday, establishing what it describes as foundational infrastructure

09:47:48 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | User message: Summarize the article below:

[
  {
    "id":151,
    "text_content":"Page title: NetApp's new AI Data Engine extends its intelligent storage infrastructure to AI workloads - SiliconANGLE\nSocial card title: NetApp's new AI Data Engine extends its intelligent storage infrastructure to AI workloads - SiliconANGLE\nSocial card description: NetApp's new AI Data Engine extends its intelligent storage infrastructure to AI workloads - SiliconANGLE\nNetApp\u2019s new AI Data Engine extends its intelligent storage infrastructure to AI workloads\nData storage pioneer NetApp Inc. is building on its vision of an \u201cintelligent data infrastructure\u201d platform that\u2019s fit for the artificial intelligence era with the launch of its new AI Data Engine.\nAnnounced at the company\u2019s annual user conference NetApp Insight 2025, it\u2019s a comprehensive service that aims to make AI data pipelines simpler, 

...

09:52:59 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | User message: Read the news item objects below, and for each, output ONE neutral sentence of ≤40 words that captures the key facts, with no labels or extra text, in the specified JSON format.
[
  {
    "id":677,
    "input_text":"Salesforce Puts Slack, Voice Tools At Center Of AI Strategy\n\nSalesforce stock is down 27% in 2025 despite its artificial intelligence push. At Dreamforce, Salesforce unveiled a new Slack-based strategy.\nThe post\u2026\n\n- Salesforce unveiled a new AI strategy at Dreamforce centered on Slack and voice-based tools, positioning collaboration and voice interaction as core product priorities (reported by Investor's Business Daily on 2025-10-13).\n- Despite the AI push, Salesforce stock is down 27% in 2025, reflecting investor skepticism; the Slack-centric approach is intended to boost engagement, monetization and competitiveness in enterprise collaboration versus rivals like Microsoft and 

09:53:05 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Result: results_list=[DistilledStory(id=656, short_summary='Yahoo Finance predicts SoundHound AI could grow 10x by 2035 but says its high valuation undermines that outlook, despite a large addressable market and rapid growth.')]
09:53:05 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | User message: Read the news item objects below, and for each, output ONE neutral sentence of ≤40 words that captures the key facts, with no labels or extra text, in the specified JSON format.
[
  {
    "id":303,
    "input_text":"Gemini 3 Just Simulated macOS in a Single HTML File \ud83e\udd2f\n\n- Gemini 3 was used to generate a simulated macOS-like user interface implemented as a single HTML file (shared via CodePen); a related Windows simulation was produced on Build with Gemini in 172 seconds; the example was posted on r\/singularity.\n- Demonstrates LLM capability to produce functional front-end code and lightwei

09:53:11 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | User message: Read the news item objects below, and for each, output ONE neutral sentence of ≤40 words that captures the key facts, with no labels or extra text, in the specified JSON format.
[
  {
    "id":23,
    "input_text":"Tonal Eyes Growth with Retail Expansion, AI-Powered Pilates\n\n- no content"
  }
]
09:53:13 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Result: results_list=[DistilledStory(id=695, short_summary='A growing number of visual-arts programs at independent art colleges and universities are launching courses that integrate AI-driven creative tools into studio curricula to prepare students for machine-assisted production methods and address practical, ethical and authorship questions.')]
09:53:13 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | User message: Read the news item objects below, and for each, output ONE neutral sentence of ≤40 words that captures th

09:53:31 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Result: results_list=[DistilledStory(id=122, short_summary='A leak indicates Google may release the Gemini 3.0 model on October 22.')]
09:53:33 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Result: results_list=[DistilledStory(id=165, short_summary='California Gov. Gavin Newsom signed a bill regulating consumer-facing AI chatbots in California, requiring companies to reassess compliance, transparency and deployment strategies and potentially increasing operational and legal costs for AI vendors while setting a regulatory precedent.')]
09:53:34 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Result: results_list=[DistilledStory(id=20, short_summary='Goldman Sachs CEO David Solomon touted AI as employee compensation jumped 14%.')]
09:53:37 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Result: results_list=[DistilledStory(id=658, short_summary='A long-time internet-

✅ Completed Step 4: Generated AI summaries for 92/92 articles
⏱️  Total execution time: 367.85s
📊 Final result:
✅ Step 4 step_04_extract_summaries completed successfully! Generated AI-powered summaries for 92/92 articles.
💾 Summaries stored in headline DataFrame.


In [7]:
# if we get a refusal, examine prompt and delete rows
bad_stuff =["What\'s the Best Way ",
            "AI could make it eas",
            "AI can design toxic ",
           ]
print([len(s) for s in bad_stuff])
headline_df = state.headline_df
headline_df.loc[(headline_df["title"].str[:20].isin(bad_stuff))]



[20, 20, 20]


,source,title,url,published,rss_summary,id,isAI,status,final_url,html_path,last_updated,text_path,content_length,domain,site_name,reputation,summary,description,tags,short_summary


In [8]:
# remove bad stuff
state.headline_df_to_dict(headline_df.loc[~headline_df["title"].str[:20].isin(bad_stuff)])


In [9]:
# User prompt to run workflow
# user_prompt = "Run step 5, Rate articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("rate_articles")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


09:53:56 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Rating 92 AI articles using fn_rate_articles
09:53:56 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Calculating article ratings for 92 articles
09:53:56 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Rating recency


▶ Starting Step 5: step_05_rate_articles


09:53:56 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Rating spam probability
09:53:56 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Initialized LangfuseClient
09:53:56 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Successfully retrieved prompt 'newsagent/rate_quality' from Langfuse
09:53:56 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Parsed prompt 'newsagent/rate_quality': model=gpt-4.1, system_len=1849, user_len=246


concurrency:  16


09:54:01 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | low quality articles: {0.0: 68, 1.0: 12, 3.442477108469977e-14: 1, 0.0001584362410990077: 1, 1.8795288165390832e-12: 1, 8.764248219443636e-08: 1, 8.25807328555592e-14: 1, 6.962258245244699e-06: 1, 0.9999998063873693: 1, 0.9997694306355145: 1, 0.49999990964071384: 1, 0.9999484309392331: 1, 0.9999987335551229: 1, 0.999997064707474: 1}
09:54:01 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Successfully retrieved prompt 'newsagent/rate_on_topic' from Langfuse
09:54:01 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Parsed prompt 'newsagent/rate_on_topic': model=gpt-4.1, system_len=1790, user_len=240


...

,id,site_name,title,bradley_terry,bt_z
67,67,WXOW News 19,Sora 2 and ChatGPT are consuming so much power that OpenAI just did another 10 gigawatt deal,12.256236,1.831753
47,47,Globe Newswire,OpenAI and Broadcom announce strategic collaboration to deploy 10 gigawatts of OpenAI-designed AI accelerators,12.256198,1.831747
39,39,Yahoo,OpenAI taps Broadcom to build its first AI processor in latest chip deal,11.705024,1.749372
13,13,TechCrunch,Google plans to invest $15B over five years in India to set up a 1GW data center and AI hub in the southern Andhra Pradesh state; Google has 14K staff in India (Jagmeet Singh/TechCrunch),11.704445,1.749285
20,20,Yahoo,Google to spend $15 billion on AI data centre in biggest India investment,10.810344,1.615657
...,...,...,...,...,...
70,70,The Washington Post,"As the AI industry booms, manufacturing falls behind",-9.589444,-1.433188
91,91,Biztoc,1 Top Artificial Intelligence (AI) Stock Ready for a Bull Run,-9.631863,-1.439528
44,44,The Wall Street Journal,Why Broadcom’s Bet on OpenAI Is a Big Risk,-9.835840,-1.470013
30,30,Bloomberg,Goldman’s Solomon Touts AI as Employee Compensation Jumps 14%,-10.347993,-1.546557


10:11:54 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Top 10 ids: [67, 47, 39, 13, 20, 23, 9, 36, 59, 74]
10:11:54 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | After round 11/19: 
10:11:54 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Number of ranking changes: 38
10:11:54 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Sum of absolute ranking changes: 1324.0 (avg rank chg 14.39)
10:11:54 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | last_two: 16.619565217391305, prev_two: 12.597826086956523
10:11:54 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Increase in avg rank change, stopping
10:11:54 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Low rated articles: 1
10:11:54 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | low rated article: 1 Top Artificial Intelligence (AI) Stock Ready for a Bull Run -0.12034500985010255


✅ Completed Step 5: Rated 91 articles


10:11:55 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Sent email with 91 rated articles
10:11:55 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Inserting 91 articles into database
10:11:56 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Successfully inserted 91/91 articles


⏱️  Total execution time: 1079.67s
📊 Final result:
✅ Step 5 step_05_rate_articles completed successfully! Rated 91 articles with average rating 6.1/10.
⭐ High quality articles (≥7.0): 34
💾 Ratings stored in persistent state.


In [10]:
headline_df=state.headline_df
with pd.option_context('display.max_columns', None, 'display.width', None, 'display.max_colwidth', None  ):
    display(headline_df.sort_values("rating", ascending=False)[['site_name', 'title', 'rating', 'short_summary']])


,site_name,title,rating,short_summary
13,TechCrunch,Google plans to invest $15B over five years in India to set up a 1GW data center and AI hub in the southern Andhra Pradesh state; Google has 14K staff in India (Jagmeet Singh/TechCrunch),10.380134,"Google will invest $15 billion through 2030 to build a 1‑gigawatt data center and AI hub in Visakhapatnam, Andhra Pradesh, in partnership with Bharti Airtel and AdaniConneX, its largest investment in India and largest outside the U.S."
9,SiliconANGLE,Oracle adds AI capabilities to core database and launches a lakehouse platform,10.111769,"Oracle announced general availability of Oracle AI Database 26ai and Autonomous AI Lakehouse, embedding native AI functions including vector search, ONNX/LLM and Apache Iceberg support, available on OCI, AWS, Azure, GCP and Exadata Cloud@Customer, with free upgrades from 23ai."
20,Yahoo,Google to spend $15 billion on AI data centre in biggest India investment,9.972912,"Google will invest $15 billion over five years to build an AI data-centre campus with an initial 1 GW capacity in Visakhapatnam, Andhra Pradesh, which Google Cloud CEO Thomas Kurian called the company's ""largest AI hub"" outside the United States."
23,Tom's Hardware,"Inside the AI accelerator arms race: AMD, Nvidia, and hyperscalers commit to annual releases through the decade",9.783504,"Major vendors and hyperscalers have adopted an annual AI-accelerator release cadence through the late 2020s, with AMD, Nvidia, Google, Amazon, Meta, Microsoft and Intel announcing roadmap chips including AMD MI430X (H2 2026) and Nvidia Rubin (late 2026)."
1,The Register,"Shadow AI: Staffers are bringing AI tools they use at home to work, warns Microsoft",9.653580,"Microsoft warns of 'shadow AI' after a UK survey found 71% of employees have used unapproved consumer AI at work, 51% still do, with 49% for workplace communications, 40% for reports, 22% for finance, and 32% citing privacy concerns."
...,...,...,...,...
85,BGR,Leak Says Google Might Release The Gemini 3.0 Model On October 22,1.331166,A leak indicates Google may release the Gemini 3.0 model on October 22.
88,Biztoc,Stock Splits Ahead? 3 Artificial Intelligence (AI) Stocks to Keep on Your Radar,1.247014,"ASML's share price could prompt a fifth stock split, Meta Platforms has never split its stock, and AppLovin is considering a split amid rapid revenue and profit growth, according to Yahoo Finance on Oct. 13, 2025."
87,We And The Color,The New Authenticity: How Motyw’s Amo’ya Branding Fuses AI with Human Emotion,1.211461,"Motyw used AI to create the Amo'ya brand, generating floral illustrations and design directions that designers vetted to build a human-centered visual identity, accelerating ideation and reducing time-to-market while enabling scalable packaging and digital assets."
89,Biztoc,10 Under-the-Radar Artificial Intelligence (AI) Stocks Worth a Closer Look,0.648174,"Under‑the‑radar AI stocks offer targeted exposure to niche technologies, often profitable or near breakeven with recurring enterprise revenue, providing alternatives to megacap valuation premiums while carrying idiosyncratic risk."


In [11]:
for row in headline_df.sort_values("rating", ascending=False).itertuples():
    display(Markdown(f"""
{row.rating:.1f} - [{row.title}]({row.url}) - {row.site_name}

{row.short_summary}
    """))
    


10.4 - [Google plans to invest $15B over five years in India to set up a 1GW data center and AI hub in the southern Andhra Pradesh state; Google has 14K staff in India (Jagmeet Singh/TechCrunch)](https://techcrunch.com/2025/10/14/google-to-invest-15b-in-indian-ai-infrastructure-hub/) - TechCrunch

Google will invest $15 billion through 2030 to build a 1‑gigawatt data center and AI hub in Visakhapatnam, Andhra Pradesh, in partnership with Bharti Airtel and AdaniConneX, its largest investment in India and largest outside the U.S.
    


10.1 - [Oracle adds AI capabilities to core database and launches a lakehouse platform](https://siliconangle.com/2025/10/14/oracle-adds-ai-capabilities-core-database-launches-lakehouse-platform/) - SiliconANGLE

Oracle announced general availability of Oracle AI Database 26ai and Autonomous AI Lakehouse, embedding native AI functions including vector search, ONNX/LLM and Apache Iceberg support, available on OCI, AWS, Azure, GCP and Exadata Cloud@Customer, with free upgrades from 23ai.
    


10.0 - [Google to spend $15 billion on AI data centre in biggest India investment](https://www.yahoo.com/news/articles/google-spend-15-billion-ai-082219753.html) - Yahoo

Google will invest $15 billion over five years to build an AI data-centre campus with an initial 1 GW capacity in Visakhapatnam, Andhra Pradesh, which Google Cloud CEO Thomas Kurian called the company's "largest AI hub" outside the United States.
    


9.8 - [Inside the AI accelerator arms race: AMD, Nvidia, and hyperscalers commit to annual releases through the decade](https://www.tomshardware.com/tech-industry/artificial-intelligence/inside-the-ai-accelerator-arms-race-amd-nvidia-and-hyperscalers-commit-to-annual-releases-through-the-decade) - Tom's Hardware

Major vendors and hyperscalers have adopted an annual AI-accelerator release cadence through the late 2020s, with AMD, Nvidia, Google, Amazon, Meta, Microsoft and Intel announcing roadmap chips including AMD MI430X (H2 2026) and Nvidia Rubin (late 2026).
    


9.7 - [Shadow AI: Staffers are bringing AI tools they use at home to work, warns Microsoft](https://www.theregister.com/2025/10/14/microsoft_warns_of_the_dangers/) - The Register

Microsoft warns of 'shadow AI' after a UK survey found 71% of employees have used unapproved consumer AI at work, 51% still do, with 49% for workplace communications, 40% for reports, 22% for finance, and 32% citing privacy concerns.
    

...


1.3 - [Leak Says Google Might Release The Gemini 3.0 Model On October 22](https://www.bgr.com/1996171/google-gemini-3-release-october-22/) - BGR

A leak indicates Google may release the Gemini 3.0 model on October 22.
    


1.2 - [Stock Splits Ahead? 3 Artificial Intelligence (AI) Stocks to Keep on Your Radar](https://biztoc.com/x/2bd674cb7350c6a3) - Biztoc

ASML's share price could prompt a fifth stock split, Meta Platforms has never split its stock, and AppLovin is considering a split amid rapid revenue and profit growth, according to Yahoo Finance on Oct. 13, 2025.
    


1.2 - [The New Authenticity: How Motyw’s Amo’ya Branding Fuses AI with Human Emotion](https://weandthecolor.com/the-new-authenticity-how-motyws-amoya-branding-fuses-ai-with-human-emotion/206214) - We And The Color

Motyw used AI to create the Amo'ya brand, generating floral illustrations and design directions that designers vetted to build a human-centered visual identity, accelerating ideation and reducing time-to-market while enabling scalable packaging and digital assets.
    


0.6 - [10 Under-the-Radar Artificial Intelligence (AI) Stocks Worth a Closer Look](https://biztoc.com/x/4a64f265947bef37) - Biztoc

Under‑the‑radar AI stocks offer targeted exposure to niche technologies, often profitable or near breakeven with recurring enterprise revenue, providing alternatives to megacap valuation premiums while carrying idiosyncratic risk.
    


0.3 - [Prediction: This Artificial Intelligence (AI) Stock Could Grow 10X by 2035](https://biztoc.com/x/1493804e2181929e) - Biztoc

Yahoo Finance predicts SoundHound AI could grow 10x by 2035 but says its high valuation undermines that outlook, despite a large addressable market and rapid growth.
    

In [12]:
# User prompt to run workflow
# user_prompt = "Run step 6, Cluster articles by topic"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("cluster_by_topic")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


10:11:56 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Starting topic extraction for clustering
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/extract_topics' from Langfuse
INFO:llm:Parsed prompt 'newsagent/extract_topics': model=gpt-5-nano, system_len=1100, user_len=80
10:11:56 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Using model 'gpt-5-nano' for topic extraction
10:11:56 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Processing 91 articles for topic extraction
10:11:56 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Initialized LLMagent:
system_prompt: 
# Role and Objective
You are an expert AI news analyst. Your task is to extract a list of up to **5** distinct topics from provided news summaries (or an empty list if no topics can be extracted).

# Input Format
You will receive a list of news summary objects in JSON format including fields "id" and "summary".

# Output Fo

▶ Starting Step 6: step_06_cluster_by_topic
concurrency:  16


10:11:56 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | User message: Extract up to 5 distinct, broad topics from the news summary below:
[
  {
    "id":10,
    "input_text":"Red Hat AI 3 targets production inference and agents\nRed Hat AI 3 targets production inference and agents - SiliconANGLE\n- Red Hat announced Red Hat AI 3, a hybrid cloud-native platform bundling OpenShift AI, Enterprise Linux AI and the AI Inference Server to manage inference across datacenters, clouds and edge; introduces llm-d, a new distributed inference engine built on open-source work like vLLM to schedule and serve LLMs on Kubernetes. Quote: \u201cAI platforms aren\u2019t going to run a single model on a single inference server on a single machine,\u201d \u2014 Joe Fernandes.\n- Business and practical impact: designed to move enterprises from experimentation to operational AI at scale, address rising generative-AI costs and let organizations serve models internally as scalable, tracked end

10:11:56 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | User message: Extract up to 5 distinct, broad topics from the news summary below:
[
  {
    "id":20,
    "input_text":"Google to spend $15 billion on AI data centre in biggest India investment\nBENGALURU (Reuters) -Google said on Tuesday it would invest $15 billion over five years to set up an artificial intelligence data centre in India's southern state of Andhra Pradesh, its biggest ever investment in the world's most populous nation.  The U.S. tech giant's plan comes amid a tense diplomatic standoff between New Delhi and Washington over tariffs and a stalled trade deal, as Prime Minister Narendra Modi has urged a boycott of foreign goods.  Indian infotech and finance ministers attended the New Delhi event at which Google Cloud CEO Thomas Kurian said the data centre in Andhra Pradesh would be the company's \"largest AI hub\" outside the United States.\n- Google will invest $15 billion over five years to build an

10:11:56 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | User message: Extract up to 5 distinct, broad topics from the news summary below:
[
  {
    "id":30,
    "input_text":"Goldman\u2019s Solomon Touts AI as Employee Compensation Jumps 14%\n- no content"
  },
  {
    "id":31,
    "input_text":"Europe Fights for AI Independence to Avoid Becoming Tech \u2018Colony\u2019\n- no content"
  },
  {
    "id":32,
    "input_text":"AI Stocks Are in a Bubble, Most Investors Say in BofA Survey\n- no content"
  },
  {
    "id":33,
    "input_text":"AI Stocks Are in a Bubble, BofA Fund Manager Survey Finds\nA global survey by Bank of America finds a record share of global fund managers saying artificial intelligence stocks are in a bubble, which was viewed as the biggest market tail risk. Elyas Galou, senior investment strategist at BofA Securities, runs through the findings of the bank's global fund manager survey.\n- Bank of America\u2019s global fund manager survey found a reco

...

/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 

=== HDBSCAN Parameters ===
min_cluster_size:   2
min_samples:        2
n_components:       33
=== Clustering Quality Metrics ===
Number of clusters: 11
Noise points: 49 (53.8%)
Average cluster size: 3.8 ± 1.3
Cluster size range: 2 - 6
=== Quality Scores ===
Silhouette Score: 0.327 (higher is better)
Calinski-Harabasz Score: 7.1 (higher is better)
Davies-Bouldin Score: 1.109 (lower is better)
HDBSCAN Validity Index: 0.117
Composite Score: 0.222 (higher is better)

[I 2025-10-14 10:15:36,651] Trial 198 finished with value: -0.22190815135458547 and parameters: {'n_components': 33, 'min_cluster_size': 2, 'min_samples': 2}. Best is trial 66 with value: -0.27757801235067286.
=== HDBSCAN Parameters ===
min_cluster_size:   2
min_samples:        2
n_components:       62
=== Clustering Quality Metrics ===
Number of clusters: 10
Noise points: 62 (68.1%)
Average cluster size: 2.9 ± 1.2
Cluster size range: 2 - 6
=== Quality Scores ===
Silhouette Score: 0.318 (higher is better)
Calinski-Harabasz Sco

10:15:52 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | 8: AI Reshaping Work, Markets, Governance
10:15:52 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Shadow AI: Staffers are bringing AI tools they use at home to work, warns Microsoft (Shadow AI, Copilot, Cybersecurity, Policy And Regulation, Governance, Privacy, Gen AI)
Morning Bid: Volatility builds as AI jockeys with trade (AI Chips, OpenAI, Semiconductor Chips, Infrastructure, Finance, Stocks, Economics)
AI will fuel populism and power shifts, JPMorgan Chase says (Policy And Regulation, Economics, Gen AI, AI Risk, Safety And Alignment, Governance, Geopolitics of AI: Decoding the New Global Operating System)
What This Year’s Nobel Prize Teaches About Innovation And AI Risk (AI, Governance, Policy And Regulation, Innovation, Gen AI, AI Risk, Ethics)
U.S. Workers Who Use AI Earn 40% More In 2025, New Report Says (future of work, AI skills, Jobs And Careers, career growth, Labor Market, Economics, A

10:17:05 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | OpenAI wants to be the next big tech platform. A developer expert weighs in on how this is going. (OpenAI, App Directory, Generative Artificial Intelligence, Developer Ecosystem, Language Models, Agents, SDK)
Harvey's founders say OpenAI is 'indirectly' its biggest competitor (OpenAI, Harvey, Law Firms, LegalTech, Foundation Models, Venture Capital, Language Models)
OpenAI's massive deals show Sam Altman is selling a vision of a world-changing product and achieving it via world-changing financial engineering to raise $1T+ ()
AI Isn't Apple's Strength, OpenAI 'First Real Competitor' To iPhone Maker, Says John Sculley (AI, OpenAI, Apple, Siri, AI Hardware, Virtual Assistants, Products)
10:17:05 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | 

10:17:16 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | 4: AI Driving Corporate Growth
10:17:16 | NewsletterAgent.test_newsletter_202510140910

⏱️  Total execution time: 381.79s
📊 Final result:
✅ Step 6 step_06_cluster_by_topic completed successfully! Organized 91 articles into topic clusters.


In [13]:
# User prompt to run workflow
# user_prompt = "Run step 7, select section topics"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("select_sections")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


10:18:17 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Free form categorization of articles
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/cat_proposal' from Langfuse
INFO:llm:Parsed prompt 'newsagent/cat_proposal': model=gpt-5-mini, system_len=638, user_len=1179
10:18:18 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Initialized LLMagent:
system_prompt: # Role & Objective
You are **“The News Pulse Analyst.”**
Your task: read a daily batch of AI-related news items and surface ** 10-30 ** short, high-impact topic titles for an executive summary.
You will receive today's AI-related news items in markdown format.
Each item will have headline, URL, topics, an item rating, and bullet-point summary.
Return ** 10-30 ** distinct, high-impact topics in the supplied JSON format.
Ensure that you propose topics that cover most of the highest-rated items (rated 7 and above)

# Input Format
Headline - Site

Rating: x.x

Topics

▶ Starting Step 7: step_07_select_sections


10:20:44 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Result: items=['Google $15B India Hub', 'Oracle AI Database Launch', 'AI Accelerator Arms Race', 'Shadow AI At Work', 'Salesforce Agentforce Expansion', 'OpenAI Broadcom 10GW Deal', 'NetApp AI Data Engine', 'AI Infrastructure Capex Surge', 'Visa Trusted Agent Protocol', 'Oracle Deploys AMD MI450', 'Nvidia DGX Spark Desktop', 'Red Hat AI 3 Platform', 'OpenAI Power Concerns', 'AI Stocks Bubble Concern', 'DirecTV Shoppable Screensavers', 'Lila Sciences $115M Raise', 'Anthropic High UK Pay', 'Cryptography-As-A-Service Partnership', 'Archive360 Azure EDiscovery', 'Flex Modular Data Centers', 'DeepTempo Cribl Security Alliance']
10:20:44 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Cleaning up initial categories: ['Flex Modular Data Centers', 'AI Driving Corporate Growth', 'DeepTempo Cribl Security Alliance', 'Oracle Deploying 50K AMD AI Chips', 'Oracle AI Database Launch', 'AI Investment vs Industria

INFO:llm:Parsed prompt 'newsagent/cat_assignment': model=gpt-5-mini, system_len=1945, user_len=176
10:22:32 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Initialized LLMagent:
system_prompt: You are an AI Topic Router.

Task

Given one news item and a list of candidate topics, output exactly one string: either the single topic whose meaning best matches the news item, or Other if no candidate fits with sufficient confidence.

Rules
	1.	Read fully. Focus on the headline/lede and main subject, not incidental mentions.
	2.	Semantic match. Compare the news item’s meaning to every candidate topic.
	3.	Choose one topic. Pick the topic with the highest semantic overlap with the news item’s main subject.
	4.	Confidence threshold. If your best match has < 60% confidence, output Other.
        - Heuristics:
        - ≥90%: The topic (or a clear synonym) is explicit and the article is primarily about it.
    	- 60–89%: Strong indirect match; the main subject clearly falls under 

10:22:32 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | User message: CANDIDATE TOPICS
AI Accelerator Arms Race
AI Advertising And Branding
AI Governance
AI Infrastructure Investment Surge
AI Infrastructure Security Partnerships
AI Investment Trends
AI Reshaping Work
AI Stock Bubble Concern
Anthropic UK Pay
Archive360 Azure EDiscovery
Big Tech Backs AI Startups
Cryptography-as-a-Service
DeepTempo Cribl Partnership
DirecTV Shoppable Screensavers
Enterprise AI Infrastructure
Flex Modular Data Centers
Google $15B India Hub
Lila Sciences $115M Raise
NetApp AI Data Engine
Nvidia Compact Supercomputers
Nvidia DGX Spark
OpenAI Broadcom 10GW Deal
OpenAI Infrastructure Expansion
OpenAI Power Concerns
Oracle AI Database Launch
Oracle Deploys 50K AMD MI450
Red Hat AI 3 Platform
Salesforce Agentforce Expansion
Shadow AI
Visa Trusted Agent Protocol

Classify the news item into exactly one of the candidate topics above. If your best match is < 60% confidence, output Other.

NEWS ITE

...

10:22:56 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | User message: Deduplicate the following news articles:
{"id":{"13":13,"20":20},"extended_summary":{"13":"Google plans to invest $15B over five years in India to set up a 1GW data center and AI hub in the southern Andhra Pradesh state; Google has 14K staff in India (Jagmeet Singh\/TechCrunch)\n\nGoogle's investment in India will roll out over the next five years, through 2030.\n\nGoogle, AI, Data Center, Infrastructure, Subsea Cables, Deals, India\n\n- Google will invest $15 billion to build a 1\u2011gigawatt data center and AI hub in Visakhapatnam, Andhra Pradesh, staged over the next five years through 2030; the facility will include subsea cable landing infrastructure, be scaled to \u201cmultiple gigawatts\u201d over time, and provide a full stack (TPUs, access to Gemini, agent\/platform tools) to support Search, YouTube, Gmail and Ads.\n- Strategic and market impact: this is Google\u2019s largest investment in 

10:23:04 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Result: results_list=[DupeRecord(id=28, dupe_id=-1), DupeRecord(id=29, dupe_id=28), DupeRecord(id=87, dupe_id=-1)]
10:23:05 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Result: results_list=[DupeRecord(id=33, dupe_id=-1), DupeRecord(id=32, dupe_id=33)]
10:23:06 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Result: results_list=[DupeRecord(id=13, dupe_id=-1), DupeRecord(id=20, dupe_id=13)]
10:23:08 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Result: results_list=[DupeRecord(id=14, dupe_id=-1), DupeRecord(id=62, dupe_id=-1)]
10:23:08 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Result: results_list=[DupeRecord(id=1, dupe_id=-1), DupeRecord(id=50, dupe_id=-1)]
10:23:09 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Result: results_list=[DupeRecord(id=15, dupe_id=-1), DupeRecord(id=18, dupe_id=-1), DupeRecord(id=35, dupe_id=-1

10:23:30 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Handling singleton categories
10:23:30 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Singleton categories (4): ['Oracle Deploys 50K AMD MI450', 'Google $15B India Hub', 'Nvidia DGX Spark', 'AI Stock Bubble Concern']
10:23:30 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Non-singleton categories (13): ['AI Reshaping Work', 'AI Governance', 'AI Investment Trends', 'Other', 'AI Infrastructure Investment Surge', 'Big Tech Backs AI Startups', 'Enterprise AI Infrastructure', 'Salesforce Agentforce Expansion', 'AI Accelerator Arms Race', 'OpenAI Infrastructure Expansion', 'Shadow AI', 'AI Advertising And Branding', 'OpenAI Broadcom 10GW Deal']
10:23:30 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Reassigning 4 singleton articles to non-singleton categories
10:23:30 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | User message: CANDIDATE TOPICS
AI Accel

10:23:35 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Result: topic_title='AI Investment Trends'
10:23:38 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Result: topic_title='Enterprise AI Infrastructure'
10:23:39 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Result: topic_title='AI Infrastructure Investment Surge'
10:23:42 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Result: topic_title='Enterprise AI Infrastructure'
10:23:42 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Reassigned 4 singleton articles
10:23:42 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Final category counts: [{'cat': 'AI Reshaping Work', 'count': 11}, {'cat': 'AI Investment Trends', 'count': 10}, {'cat': 'AI Governance', 'count': 10}, {'cat': 'Other', 'count': 7}, {'cat': 'AI Infrastructure Investment Surge', 'count': 6}, {'cat': 'Enterprise AI Infrastructure', 'count': 5}, {'cat': 'Big Tech Backs AI Startup

✅ Completed Step 7: Categories and article counts:
cat
AI Accelerator Arms Race               3
AI Advertising And Branding            2
AI Governance                         10
AI Infrastructure Investment Surge     6
AI Investment Trends                  10
AI Reshaping Work                     11
Big Tech Backs AI Startups             4
Enterprise AI Infrastructure           5
OpenAI Broadcom 10GW Deal              2
OpenAI Infrastructure Expansion        2
Other                                  7
Salesforce Agentforce Expansion        3
Shadow AI                              2
⏱️  Total execution time: 324.77s
📊 Final result:
Categories and article counts:
cat
AI Accelerator Arms Race               3
AI Advertising And Branding            2
AI Governance                         10
AI Infrastructure Investment Surge     6
AI Investment Trends                  10
AI Reshaping Work                     11
Big Tech Backs AI Startups             4
Enterprise AI Infrastructure           5

In [14]:
# User prompt to run workflow
# user_prompt = "Run step 8, draft sections"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("draft_sections")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


10:23:42 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Step 8a: Selecting stories from 67 total
10:23:42 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | 12 must-include stories with rating > 8
10:23:42 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | 55 stories with rating <= 8


▶ Starting Step 8: step_08_draft_sections


10:23:45 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Selected 55 diverse Tier 2 stories via MMR
10:23:45 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Total selected stories: 67 (target: 100)
10:23:45 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Drafting sections for 13 categories (including Other) from selected stories
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/write_section' from Langfuse
INFO:llm:Parsed prompt 'newsagent/write_section': model=gpt-5, system_len=1854, user_len=22
10:23:45 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Initialized LLMagent:
system_prompt: You are a newsletter editor transforming a collection of raw news stories into a compelling, coherent topic summary.

# TASK
Transform the list of news stories into a well-structured newsletter section with a strong title, crisp headlines, and punchy summaries.

# INPUT
- a list of json objects sorted b

10:23:45 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | User message: STORIES:
[{"id":13,"rating":11.3801335966,"summary":"Google will invest $15 billion through 2030 to build a 1\u2011gigawatt data center and AI hub in Visakhapatnam, Andhra Pradesh, in partnership with Bharti Airtel and AdaniConneX, its largest investment in India and largest outside the U.S.","site_name":"TechCrunch","url":"https:\/\/techcrunch.com\/2025\/10\/14\/google-to-invest-15b-in-indian-ai-infrastructure-hub\/"},{"id":8,"rating":9.1774199337,"summary":"Market volatility spiked as the VIX hit its highest since mid-June, Broadcom rose 10% after partnering with OpenAI on in-house AI processors, Google pledged $15 billion for Indian datacenters, and Citigroup forecast $2.8 trillion in AI infrastructure spending through 2029.","site_name":"Reuters","url":"https:\/\/www.reuters.com\/business\/finance\/global-markets-view-usa-2025-10-14\/"},{"id":74,"rating":6.339085874,"summary":"China Mobile announ

10:23:46 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | User message: STORIES:
[{"id":39,"rating":11.3432764445,"summary":"OpenAI will design custom AI processors that Broadcom will develop and deploy starting in H2 2026, targeting 10 gigawatts of capacity by end\u20112029, with power draw roughly equivalent to over eight million U.S. households.","site_name":"Yahoo","url":"https:\/\/finance.yahoo.com\/news\/openai-taps-broadcom-build-first-130111782.html"},{"id":44,"rating":4.2094937477,"summary":"Broadcom's bet on OpenAI exposes the company to concentrated strategic and financial risk.","site_name":"The Wall Street Journal","url":"https:\/\/www.wsj.com\/finance\/investing\/why-broadcoms-bet-on-openai-is-a-big-gamble-f6631ced"}]
10:23:46 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | User message: STORIES:
[{"id":17,"rating":7.9668126079,"summary":"Harvey, an AI startup for law firms built on OpenAI models and backed by the OpenAI Startup Fund, has ra

...

10:30:24 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO |   Other: Coherence=2.0, Quality=3.0, Iterate=True
10:30:24 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO |   AI Governance: Coherence=5.5, Quality=6.0, Iterate=True
10:30:24 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO |   AI Infrastructure Investment Surge: Coherence=8.0, Quality=7.0, Iterate=True
10:30:24 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO |   AI Investment Trends: Coherence=8.5, Quality=7.5, Iterate=False
10:30:24 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO |   Salesforce Agentforce Expansion: Coherence=9.0, Quality=8.0, Iterate=True
10:30:24 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO |   OpenAI Broadcom 10GW Deal: Coherence=9.5, Quality=8.5, Iterate=False
10:30:24 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO |   Shadow AI: Coherence=7.5, Quality=7.2, Iterate=False
10:30:24 | NewsletterAgent.test_ne

10:30:25 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO |       New: Nobel economists warn AI market concentration threatens innovation-led growth
10:30:25 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO |     REWRITE id=46: Directly supports global power-shift narrative; headline is long—sharpen and keep active voice.
10:30:25 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO |       Old: JPMorgan warns AI could fuel populism and shift global power; urges strategy and governance overhauls
10:30:25 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO |       New: JPMorgan warns AI could fuel populism and shift global power, urges governance overhaul
10:30:25 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO |     REWRITE id=31: Clear geopolitical fit (AI sovereignty). Minor wording tweak for concision.
10:30:25 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO |       Old: Europe races for AI independence to cut 

10:30:25 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO |       Old: Salesforce brings GPT-5 and Claude to Agentforce, Slack, and Tableau; adds Agentforce Commerce and secure‑cloud Claude for regulated sectors.
10:30:25 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO |       New: Salesforce adds GPT‑5 and Claude to Agentforce, Slack, and Tableau; launches Agentforce Commerce, secure Claude for regulated sectors.
10:30:25 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO |     REWRITE id=77: Clarify and simplify awkward construction (“Slack- and voice-led”); keep active voice and concrete feature list.
10:30:25 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO |       Old: Salesforce unveils Slack- and voice-led AI features at Dreamforce: voice meetings, AI summaries, and workflow automation.
10:30:25 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO |       New: Salesforce debuts Slack and voice AI at Dreamforce: voice me

10:30:25 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO |       New: AMD, Nvidia and hyperscalers commit to annual AI accelerators through the late 2020s, including AMD MI430X and Nvidia Rubin in 2026
10:30:25 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO |     REWRITE id=72: Relevant to chip demand and market access; headline can be clearer and more concise.
10:30:25 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO |       Old: Nvidia gets $2B xAI order; U.S. approves UAE chip exports, opening Middle East GPU sales
10:30:25 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO |       New: Nvidia secures $2B xAI investment and U.S. approval to ship GPUs to the UAE, boosting Middle East demand
10:30:25 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO |     MOVE id=68 to Other: Commentary about Apple vs OpenAI doesn’t fit the chip-race narrative.
10:30:25 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO |   Bi

✅ Completed Step 8: Drafted 46 stories across 13 sections
⏱️  Total execution time: 402.48s
📊 Final result:
Drafted 46 stories across 13 sections


In [15]:
newsletter_section_df = state.newsletter_section_df.sort_values(["sort_order", "rating"],ascending=[True, False])

newsletter_section_df

,cat,section_title,id,headline,rating,prune,links,sort_order
31,AI Accelerator Arms Race,AI Chip Race Heats Up,5,Nvidia unveils GB10 ‘DGX Spark’ workstation sh...,10.984035,False,[The Register](https://www.theregister.com/202...,0
36,AI Accelerator Arms Race,AI Chip Race Heats Up,23,"AMD, Nvidia and hyperscalers commit to annual ...",9.783504,False,[Tom's Hardware](https://www.tomshardware.com/...,0
37,AI Accelerator Arms Race,AI Chip Race Heats Up,72,Nvidia secures $2B xAI investment and U.S. app...,6.410123,False,[Seeking Alpha](https://seekingalpha.com/artic...,0
10,AI Accelerator Arms Race,AI Chip Race Heats Up,81,TSMC Q3 profit jumps 28% on AI demand; warns U...,5.577872,False,[Biztoc](https://biztoc.com/x/9b44eaa976db33c5),0
8,AI Infrastructure Investment Surge,Global AI Infrastructure Arms Race,13,Google to invest $15B in 1GW AI data center hu...,11.380134,False,[TechCrunch](https://techcrunch.com/2025/10/14...,1
7,AI Infrastructure Investment Surge,Global AI Infrastructure Arms Race,8,AI infrastructure spending could reach $2.8T b...,9.177420,False,[Reuters](https://www.reuters.com/business/fin...,1
9,AI Infrastructure Investment Surge,Global AI Infrastructure Arms Race,74,China Mobile to use domestic chips to build Ch...,6.339086,False,[Biztoc](https://biztoc.com/x/1bcaee8727f716e2),1
12,AI Infrastructure Investment Surge,Global AI Infrastructure Arms Race,62,OpenAI signs letter of intent for $25B 'Starga...,5.945845,False,[International Business Times](https://www.ibt...,1
11,AI Infrastructure Investment Surge,Global AI Infrastructure Arms Race,86,Nvidia projects multi-year AI capex surge as d...,3.246035,False,[Biztoc](https://biztoc.com/x/4912d847bcdb0ecc),1
17,AI Investment Trends,AI Markets: Boom or Bubble?,33,BofA: Fund managers call AI stocks a bubble an...,8.736582,False,[Yahoo](https://finance.yahoo.com/video/ai-sto...,2


In [16]:
Markdown(state.display_newsletter().replace("$","\\\$"))


## AI Chip Race Heats Up
- Nvidia unveils GB10 ‘DGX Spark’ workstation shipping Oct 15; from Nvidia and OEMs, starting around \\$3,000, 1 petaFLOP FP4, 128GB unified memory - [The Register](https://www.theregister.com/2025/10/13/nvidia_gb10_spark/)
- AMD, Nvidia and hyperscalers commit to annual AI accelerators through the late 2020s, including AMD MI430X and Nvidia Rubin in 2026 - [Tom's Hardware](https://www.tomshardware.com/tech-industry/artificial-intelligence/inside-the-ai-accelerator-arms-race-amd-nvidia-and-hyperscalers-commit-to-annual-releases-through-the-decade)
- Nvidia secures \\$2B xAI investment and U.S. approval to ship GPUs to the UAE, boosting Middle East demand - [Seeking Alpha](https://seekingalpha.com/article/4829518-nvidia-ai-dominance-isnt-slowing-just-got-2-massive-tailwinds)
- TSMC Q3 profit jumps 28% on AI demand; warns U.S. tariffs may hit shipments and margins - [Biztoc](https://biztoc.com/x/9b44eaa976db33c5)
## Global AI Infrastructure Arms Race
- Google to invest \\$15B in 1GW AI data center hub in Visakhapatnam with Airtel, AdaniConneX by 2030 - [TechCrunch](https://techcrunch.com/2025/10/14/google-to-invest-15b-in-indian-ai-infrastructure-hub/)
- AI infrastructure spending could reach \\$2.8T by 2029, Reuters reports - [Reuters](https://www.reuters.com/business/finance/global-markets-view-usa-2025-10-14/)
- China Mobile to use domestic chips to build China’s largest AI compute network by 2028, tripling capacity - [Biztoc](https://biztoc.com/x/1bcaee8727f716e2)
- OpenAI signs letter of intent for \\$25B 'Stargate Argentina' AI data center with Sur Energy, targeting up to 500 MW - [International Business Times](https://www.ibtimes.co.uk/openai-plan-drop-25bn-argentina-bold-bet-ais-future-1747589)
- Nvidia projects multi-year AI capex surge as data centers refresh aging GPUs - [Biztoc](https://biztoc.com/x/4912d847bcdb0ecc)
## AI Markets: Boom or Bubble?
- BofA: Fund managers call AI stocks a bubble and top tail risk - [Yahoo](https://finance.yahoo.com/video/ai-stocks-bubble-bofa-fund-110412556.html)
- China approves Pony.ai and WeRide for Hong Kong IPOs - [Bloomberg](https://www.bloomberg.com/news/articles/2025-10-14/pony-ai-weride-get-chinese-regulator-s-nod-to-list-in-hong-kong)
- Nvidia secures \\$2B xAI investment and U.S. approval to ship GPUs to the UAE, boosting Middle East demand - [Seeking Alpha](https://seekingalpha.com/article/4829518-nvidia-ai-dominance-isnt-slowing-just-got-2-massive-tailwinds)
- OpenAI explores deals totaling over \\$1T to expand product strategy - [Bloomberg](https://www.bloomberg.com/opinion/newsletters/2025-10-13/openai-keeps-doing-deals)
- Report: UK lacks growth capital for AI and quantum - [Bloomberg](https://www.bloomberg.com/news/articles/2025-10-14/uk-lacks-growth-funding-for-critical-sectors-like-quantum-ai)
- Resistant AI raises \\$25M Series B, hits breakeven, expands fraud detection and threat intelligence - [Finextra](https://www.finextra.com/pressarticle/107466/resistant-ai-raises-25m-in-series-b-funding)
## Clouds Race for AI Compute
- Oracle Cloud to deploy 50,000 AMD MI450 GPUs in Q3 2026; Helios racks arrive in 2027 - [Yahoo](https://ca.news.yahoo.com/oracle-offer-cloud-services-using-123153641.html)
- Informatica deepens Oracle Cloud partnership: agentic AI blueprint, MCP server in IDMC by April, native MDM SaaS, Dedicated Region support - [SiliconANGLE](https://siliconangle.com/2025/10/14/informatica-expands-oracle-cloud-offerings-streamline-ai-development/)
## AI Payoffs and Pitfalls
- U.S. workers who use AI earn 40% more; only 27% received formal training - [Forbes](https://www.forbes.com/sites/carolinecastrillon/2025/10/13/us-workers-who-use-ai-earn-40-more-in-2025-new-report-says/)
- 40% of employees say AI creates 'workslop,' hurting productivity; calls grow for quality controls and training - [Fortune](https://fortune.com/2025/10/13/workslop-wasting-workers-time-ai-resentment/)
- VC replaces five analysts with AI to run \\$75 million fund, automating deal flow, diligence, and portfolio management - [The Economic Times](https://m.economictimes.com/magazines/panache/ai-to-now-manage-rs-6000-crore-fund-all-analysts-fired-is-the-future-of-high-skilled-human-jobs-at-risk/articleshow/124522369.cms)
- Abu Dhabi’s AI71 deploys production AI in healthcare and construction, automating billing, compliance and speeding permits - [Computer Weekly](https://www.computerweekly.com/news/366632644/Interview-Shaping-the-future-of-AI-in-the-UAE)
- DevOps guide to automating Claude Code setup with CLI tricks, MCP servers, container prep, and permissions hardening - [Hacker Noon](https://hackernoon.com/claude-code-setup-the-devops-guide-nobody-warned-you-about)
- Goldman Sachs cites AI progress as employee compensation rises 14% - [Bloomberg](https://www.bloomberg.com/news/articles/2025-10-14/goldman-s-solomon-touts-ai-as-employee-compensation-jumps-14)
- Fetii debuts NeuralShuttle: AI predicts event surges and auto-dispatches 15-seat vans - [Forbes](https://www.forbes.com/sites/edgarsten/2025/10/13/new-fetii-rideshare-shuttle-uses-ai-to-predict-where-when-to-show-up/)
## AI Power Shift
- Nobel economists warn AI market concentration threatens innovation-led growth - [Forbes](https://www.forbes.com/sites/paulocarvao/2025/10/13/what-this-years-nobel-prize-teaches-about-innovation-and-ai-risk/)
- JPMorgan warns AI could fuel populism and shift global power, urges governance overhaul - [Axios](https://www.axios.com/2025/10/14/jpmorgan-ai-global-politics-report)
- Europe races for AI independence to reduce reliance on the U.S. and China - [Bloomberg](https://www.bloomberg.com/news/articles/2025-10-14/europe-aims-for-ai-independence-to-avoid-reliance-on-us-and-china)
- DOE worker loses clearance after uploading 187,000 porn images, including some AI-generated, to government network - [404 Media](https://www.404media.co/man-stores-ai-generated-robot-porn-on-his-government-computer-loses-access-to-nuclear-secrets/)
- DOE worker loses clearance after uploading 187,000 porn images, including some AI-generated, to government network - [404 Media](https://www.404media.co/man-stores-ai-generated-robot-porn-on-his-government-computer-loses-access-to-nuclear-secrets/)
## Salesforce AI Everywhere
- Salesforce adds GPT‑5 and Claude to Agentforce, Slack, and Tableau; launches Agentforce Commerce, secure Claude for regulated sectors. - [Yahoo](https://finance.yahoo.com/news/salesforce-deepens-ai-ties-openai-124533128.html)
- Salesforce debuts Slack and voice AI at Dreamforce: voice meetings, summaries, and workflow automation. - [Biztoc](https://biztoc.com/x/622023a0a2748b53)
- Salesforce launches Agentforce Voice for branded virtual agents and IVR with customizable speech responses. - [Biztoc](https://biztoc.com/x/51d685d08f1385e2)
## OpenAI-Broadcom Big Chip Bet
- OpenAI picks Broadcom to build custom AI chips; deployments start in late 2026, 10 GW target by 2029 - [Yahoo](https://finance.yahoo.com/news/openai-taps-broadcom-build-first-130111782.html)
- Broadcom’s OpenAI chip bet concentrates strategic and financial risk - [The Wall Street Journal](https://www.wsj.com/finance/investing/why-broadcoms-bet-on-openai-is-a-big-gamble-f6631ced)
## OpenAI's Platform and Power Plays
- OpenAI launches app directory, SDK, and coding agent to court developers for its conversational platform - [Business Insider](https://www.businessinsider.com/openai-next-big-tech-platform-developers-apple-google-devpost-2025-10)
- OpenAI signs letter of intent for \\$25B 'Stargate Argentina' AI data center with Sur Energy, targeting up to 500 MW - [International Business Times](https://www.ibtimes.co.uk/openai-plan-drop-25bn-argentina-bold-bet-ais-future-1747589)
## AI Money Moves
- Harvey AI raises \\$800M at ~\\$5B valuation; founders call OpenAI their biggest indirect rival in legal tech. - [Business Insider](https://www.businessinsider.com/harvey-versus-openai-chatgpt-for-lawyers-2025-10)
- Apple nears Prompt AI deal, including Seemour, to bolster HomeKit, iPhone imaging, and Vision Pro. - [Android Headlines](https://www.androidheadlines.com/2025/10/apple-nears-prompt-ai-acquisition-beating-elon-musks-xai.html)
- Wayve in talks with Microsoft and SoftBank to raise up to \\$2B, FT says. - [Biztoc](https://biztoc.com/x/7e7490d4c05ad822)
## Shadow AI’s Real Consequences
- Microsoft flags 'shadow AI' in UK: 71% used unapproved tools; 51% still do - [The Register](https://www.theregister.com/2025/10/14/microsoft_warns_of_the_dangers/)
## AI Fuels Ad Business
- Publicis raises growth outlook as advertising and AI demand strengthen - [Bloomberg](https://www.bloomberg.com/news/articles/2025-10-14/publicis-raises-growth-outlook-on-strong-ads-and-ai-demand)
## Other News
- California enacts child-protection rules for AI chatbots - [CNET](https://www.cnet.com/videos/apple-tv-plus-drops-the-plus-california-signs-new-ai-regs-into-law-and-amazon-customers-are-upset-about-ads-tech-today/)
- AI mapping reveals hidden ring fault driving Campi Flegrei quakes, logging 54,000 events since 2022 - [Live Science](https://www.livescience.com/planet-earth/volcanos/ai-reveals-hidden-ring-fault-that-is-unleashing-earthquakes-at-italys-campi-flegrei-volcano)
- Microsoft launches MAI-Image-1, a faster photorealistic generator coming to Copilot and Bing - [BGR](https://www.bgr.com/1996135/microsoft-ai-image-generator-launch/)
- One in three Americans use AI chatbots for health questions, not for diagnoses - [The Washington Post](https://www.washingtonpost.com/opinions/2025/10/14/ai-chatbots-health-care-diagnoses/)
- Tonal returns to growth, expands retail, adds AI-powered Reformer Pilates - [Bloomberg](https://www.bloomberg.com/news/articles/2025-10-14/tonal-adds-reformer-pilates-as-it-expands-retail-presence-returns-to-growth)


In [17]:
# User prompt to run workflow
# user_prompt = "Run step 9, finalize newsletter"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("finalize_newsletter")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


10:30:25 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Step 9: write title
10:30:25 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Compiled 46 items into markdown input
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/generate_newsletter_title' from Langfuse
INFO:llm:Parsed prompt 'newsagent/generate_newsletter_title': model=gpt-5, system_len=1082, user_len=225
10:30:25 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Initialized LLMagent:
system_prompt: You are an expert newsletter editor specializing in crafting compelling titles for technology newsletters.

Your task is to read the full newsletter content and create a factual, thematic title that captures the day's major themes.

Title Guidelines:
- 6-12 words maximum
- Factual and informative
- Summarizes 2-3 major themes from the day's news
- Use semicolons to separate distinct, unrelated themes (like a list)
- Use conjunctions like "as", "while

▶ Starting Step 9: step_09_finalize_newsletter


10:30:54 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Result: value='AI Chip Race Intensifies; Data Center Capex Soars; Bubble Concerns Mount'
10:30:54 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Generated title: AI Chip Race Intensifies; Data Center Capex Soars; Bubble Concerns Mount
10:30:54 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Step 9b: Starting critic-optimizer loop
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/critique_newsletter' from Langfuse
INFO:llm:Parsed prompt 'newsagent/critique_newsletter': model=gpt-5, system_len=2758, user_len=647
10:30:54 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Initialized LLMagent:
system_prompt: You are an expert newsletter editor with 15+ years of experience critiquing technology publications. Your role is to analyze a newsletter's quality and provide scoring and comprehensive, actionable feedbacks and instructions to edit

10:30:54 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Step 9b: Iteration 1/2 - Running critique
10:30:54 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | User message: Critique this newsletter draft:

AI Chip Race Intensifies; Data Center Capex Soars; Bubble Concerns Mount

# AI Chip Race Heats Up

- Nvidia unveils GB10 ‘DGX Spark’ workstation shipping Oct 15; from Nvidia and OEMs, starting around $3,000, 1 petaFLOP FP4, 128GB unified memory - [The Register](https://www.theregister.com/2025/10/13/nvidia_gb10_spark/)
- AMD, Nvidia and hyperscalers commit to annual AI accelerators through the late 2020s, including AMD MI430X and Nvidia Rubin in 2026 - [Tom's Hardware](https://www.tomshardware.com/tech-industry/artificial-intelligence/inside-the-ai-accelerator-arms-race-amd-nvidia-and-hyperscalers-commit-to-annual-releases-through-the-decade)
- Nvidia secures $2B xAI investment and U.S. approval to ship GPUs to the UAE, boosting Middle East demand - [Seek

10:33:23 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Result: overall_score=7.2 title_quality=8.5 structure_quality=6.5 section_quality=6.8 headline_quality=7.2 should_iterate=True critique_text="Top recommendations (prioritized by impact)\n1) Fix duplicates and redundancies\n- AI Markets: Boom or Bubble? → Remove “Nvidia secures $2B xAI investment…” (keep it only in AI Chip Race Heats Up).\n- OpenAI's Platform and Power Plays → Remove “OpenAI signs letter of intent for $25B 'Stargate Argentina'…” (keep it only in Global AI Infrastructure Arms Race).\n- AI Power Shift → Delete the duplicated DOE worker headline (it appears twice in the same section).\n\n2) Normalize structure and hierarchy\n- Add a leading “#” to the newsletter title line.\n- Consolidate or move one‑off sections (those with only one item) so every section has at least 2 stories (except Other News).\n- Merge closely related small sections to reduce fragmentation and improve flow.\n\n3) Rebalance secti

...

10:35:59 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Result: value="# AI Chips Heat Up as Data Center Capex Surges, Bubble Fears Rise\n\n## AI Chips and Accelerators Heat Up\n\n- Nvidia unveils GB10 ‘DGX Spark’ workstation, shipping Oct 15 with 1 PFLOP FP4 and 128GB unified memory - [The Register](https://www.theregister.com/2025/10/13/nvidia_gb10_spark/)\n- AMD, Nvidia, and hyperscalers commit to annual AI accelerator releases through the late 2020s - [Tom's Hardware](https://www.tomshardware.com/tech-industry/artificial-intelligence/inside-the-ai-accelerator-arms-race-amd-nvidia-and-hyperscalers-commit-to-annual-releases-through-the-decade)\n- OpenAI picks Broadcom to build custom AI chips, deployments in late 2026 with 10 GW target by 2029 - [Yahoo](https://finance.yahoo.com/news/openai-taps-broadcom-build-first-130111782.html)\n- Broadcom’s OpenAI chip bet concentrates strategic and financial risk - [The Wall Street Journal](https://www.wsj.com/finance/investing

10:35:59 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Iteration 1: Optimizer completed, draft updated
10:35:59 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Step 9b: Iteration 2/2 - Running critique
10:35:59 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | User message: Critique this newsletter draft:

# AI Chips Heat Up as Data Center Capex Surges, Bubble Fears Rise

## AI Chips and Accelerators Heat Up

- Nvidia unveils GB10 ‘DGX Spark’ workstation, shipping Oct 15 with 1 PFLOP FP4 and 128GB unified memory - [The Register](https://www.theregister.com/2025/10/13/nvidia_gb10_spark/)
- AMD, Nvidia, and hyperscalers commit to annual AI accelerator releases through the late 2020s - [Tom's Hardware](https://www.tomshardware.com/tech-industry/artificial-intelligence/inside-the-ai-accelerator-arms-race-amd-nvidia-and-hyperscalers-commit-to-annual-releases-through-the-decade)
- OpenAI picks Broadcom to build custom AI chips, deployments in l

10:37:52 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Result: overall_score=7.9 title_quality=8.5 structure_quality=8.6 section_quality=7.8 headline_quality=7.6 should_iterate=True critique_text='Top recommendations (highest impact first)\n- Consolidate overlapping Salesforce items in Enterprise AI Platforms to remove redundancy and tighten the section.\n- Reassign misfiled funding item(s) to AI Money Moves to maintain thematic coherence.\n- Tighten and clarify longer, multi‑clause headlines; keep to one clear idea each and ≤25 words.\n- Reorder a few sections’ top stories to prioritize highest‑value, market‑moving items first.\n- Consider a more engaging title for Enterprise AI Platforms and small title tweaks for consistency and tone.\n- Ensure consistent sentence case, punctuation, and hyphen/en dash usage across headlines.\n\nSpecific issues, problems, and actionable edits\n1) Section: Enterprise AI Platforms — multiple Salesforce headlines\n- Problem: Redundant/

10:37:52 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Iteration 2: Overall score = 7.9/10
10:37:52 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO |   Title: 8.5, Structure: 8.6, Section: 7.8, Headline: 7.6
10:37:52 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Iteration 2: Should iterate = True
10:37:52 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | ⚠️  Max iterations 2 reached. Final score: 7.9/10
10:37:52 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Step 9b: Critic-optimizer loop complete after 2 iteration(s). Final score: 7.9/10
10:37:52 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Step 9c complete: Final newsletter 782 words
10:37:52 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | Step 9d: Validating and sending newsletter
10:37:53 | NewsletterAgent.test_newsletter_20251014091057369527 | INFO | ✅ Sent newsletter email with subject: AI News for October 14, 2025 

✅ Completed Step 9: Finalized newsletter (782 words)
⏱️  Total execution time: 448.13s
📊 Final result:
🎉 Step 9 step_09_finalize_newsletter completed successfully!
⭐ Final quality score: 7.9/10
🔄 Completed 2 critic-optimizer iteration(s)
📰 Newsletter stored in persistent state and emailed
✅ Workflow complete! All 9 steps finished successfully.


In [20]:
display(Markdown(state.final_newsletter.replace("$", "\\\$")))

# AI Chips Heat Up as Data Center Capex Surges, Bubble Fears Rise

## AI Chips and Accelerators Heat Up

- Nvidia unveils GB10 ‘DGX Spark’ workstation, shipping Oct 15 with 1 PFLOP FP4 and 128GB unified memory - [The Register](https://www.theregister.com/2025/10/13/nvidia_gb10_spark/)
- AMD, Nvidia, and hyperscalers commit to annual AI accelerator releases through the late 2020s - [Tom's Hardware](https://www.tomshardware.com/tech-industry/artificial-intelligence/inside-the-ai-accelerator-arms-race-amd-nvidia-and-hyperscalers-commit-to-annual-releases-through-the-decade)
- OpenAI picks Broadcom to build custom AI chips, deployments in late 2026 with 10 GW target by 2029 - [Yahoo](https://finance.yahoo.com/news/openai-taps-broadcom-build-first-130111782.html)
- Broadcom’s OpenAI chip bet concentrates strategic and financial risk - [The Wall Street Journal](https://www.wsj.com/finance/investing/why-broadcoms-bet-on-openai-is-a-big-gamble-f6631ced)
- Nvidia gains \\$2B xAI investment and U.S. nod to ship GPUs to the UAE - [Seeking Alpha](https://seekingalpha.com/article/4829518-nvidia-ai-dominance-isnt-slowing-just-got-2-massive-tailwinds)
- TSMC Q3 profit up 28% on AI demand, warns U.S. tariffs may hit shipments and margins - [Biztoc](https://biztoc.com/x/9b44eaa976db33c5)

## Global AI Infrastructure Build-Out

- AI infrastructure spending could hit \\$2.8T by 2029 - [Reuters](https://www.reuters.com/business/finance/global-markets-view-usa-2025-10-14/)
- Google to invest \\$15B in a 1 GW AI data center hub in Visakhapatnam by 2030 with Airtel and AdaniConneX - [TechCrunch](https://techcrunch.com/2025/10/14/google-to-invest-15b-in-indian-ai-infrastructure-hub/)
- Oracle Cloud to deploy 50,000 AMD MI450 GPUs in Q3 2026, Helios racks in 2027 - [Yahoo](https://ca.news.yahoo.com/oracle-offer-cloud-services-using-123153641.html)
- China Mobile to build the nation’s largest AI compute network using domestic chips by 2028, tripling capacity - [Biztoc](https://biztoc.com/x/1bcaee8727f716e2)
- OpenAI signs LOI for \\$25B ‘Stargate Argentina’ AI data center with Sur Energy, targeting up to 500 MW - [International Business Times](https://www.ibtimes.co.uk/openai-plan-drop-25bn-argentina-bold-bet-ais-future-1747589)
- Nvidia projects multi‑year AI capex surge as data centers refresh aging GPUs - [Biztoc](https://biztoc.com/x/4912d847bcdb0ecc)
- Informatica expands Oracle Cloud tie‑up with agentic AI blueprint, MCP server in IDMC, native MDM SaaS, Dedicated Region support - [SiliconANGLE](https://siliconangle.com/2025/10/14/informatica-expands-oracle-cloud-offerings-streamline-ai-development/)

## AI Markets: Boom or Bubble?

- BofA: Fund managers cite AI stocks as bubble risk and top tail risk - [Yahoo](https://finance.yahoo.com/video/ai-stocks-bubble-bofa-fund-110412561.html)
- Goldman Sachs cites AI progress as employee compensation rises 14% - [Bloomberg](https://www.bloomberg.com/news/articles/2025-10-14/goldman-s-solomon-touts-ai-as-employee-compensation-jumps-14)
- China approves Pony.ai and WeRide for Hong Kong IPOs - [Bloomberg](https://www.bloomberg.com/news/articles/2025-10-14/pony-ai-weride-get-chinese-regulator-s-nod-to-list-in-hong-kong)
- OpenAI explores deals totaling over \\$1T to expand product strategy - [Bloomberg](https://www.bloomberg.com/opinion/newsletters/2025-10-13/openai-keeps-doing-deals)
- UK lacks growth capital for AI and quantum, report finds - [Bloomberg](https://www.bloomberg.com/news/articles/2025-10-14/uk-lacks-growth-funding-for-critical-sectors-like-quantum-ai)
- Publicis raises growth outlook on stronger advertising and AI demand - [Bloomberg](https://www.bloomberg.com/news/articles/2025-10-14/publicis-raises-growth-outlook-on-strong-ads-and-ai-demand)
- Resistant AI raises \\$25M Series B, hits breakeven, expands fraud detection and threat intelligence - [Finextra](https://www.finextra.com/pressarticle/107466/resistant-ai-raises-25m-in-series-b-funding)

## Enterprise AI Platforms

- Salesforce adds GPT‑5 and Claude to Agentforce, Slack, and Tableau, launches Agentforce Commerce and secure Claude for regulated sectors - [Yahoo](https://finance.yahoo.com/news/salesforce-deepens-ai-ties-openai-124533128.html)
- Salesforce debuts Slack and voice AI at Dreamforce for meetings, summaries, and workflow automation - [Biztoc](https://biztoc.com/x/622023a0a2748b53)
- Salesforce launches Agentforce Voice for branded virtual agents and IVR with customizable speech - [Biztoc](https://biztoc.com/x/51d685d08f1385e2)
- OpenAI launches app directory, SDK, and coding agent to court platform developers - [Business Insider](https://www.businessinsider.com/openai-next-big-tech-platform-developers-apple-google-devpost-2025-10)

## AI Payoffs and Pitfalls

- U.S. workers who use AI earn 40% more, but only 27% received formal training - [Forbes](https://www.forbes.com/sites/carolinecastrillon/2025/10/13/us-workers-who-use-ai-earn-40-more-in-2025-new-report-says/)
- 40% of employees say AI creates ‘workslop’ that hurts productivity, fueling calls for quality controls and training - [Fortune](https://fortune.com/2025/10/13/workslop-wasting-workers-time-ai-resentment/)
- Microsoft flags ‘shadow AI’ in the UK: 71% used unapproved tools, 51% still do - [The Register](https://www.theregister.com/2025/10/14/microsoft_warns_of_the_dangers/)
- VC automates \\$75M fund operations with AI, cutting five analyst roles - [The Economic Times](https://m.economictimes.com/magazines/panache/ai-to-now-manage-rs-6000-crore-fund-all-analysts-fired-is-the-future-of-high-skilled-human-jobs-at-risk/articleshow/124522369.cms)
- Abu Dhabi’s AI71 deploys production AI in healthcare and construction for billing, compliance, and faster permits - [Computer Weekly](https://www.computerweekly.com/news/366632644/Interview-Shaping-the-future-of-AI-in-the-UAE)
- Fetii launches NeuralShuttle to predict event surges and auto‑dispatch 15‑seat vans - [Forbes](https://www.forbes.com/sites/edgarsten/2025/10/13/new-fetii-rideshare-shuttle-uses-ai-to-predict-where-when-to-show-up/)

## AI Power Shift

- Nobel economists warn AI market concentration threatens innovation‑led growth - [Forbes](https://www.forbes.com/sites/paulocarvao/2025/10/13/what-this-years-nobel-prize-teaches-about-innovation-and-ai-risk/)
- JPMorgan warns AI could fuel populism and shift global power, urges governance overhaul - [Axios](https://www.axios.com/2025/10/14/jpmorgan-ai-global-politics-report)
- Europe races for AI independence to reduce reliance on the U.S. and China - [Bloomberg](https://www.bloomberg.com/news/articles/2025-10-14/europe-aims-for-ai-independence-to-avoid-reliance-on-us-and-china)

## AI Money Moves

- Harvey AI raises \\$800M at ~\\$5B valuation, founders call OpenAI their biggest indirect rival - [Business Insider](https://www.businessinsider.com/harvey-versus-openai-chatgpt-for-lawyers-2025-10)
- Apple nears Prompt AI deal, including Seemour, to bolster HomeKit, iPhone imaging, and Vision Pro - [Android Headlines](https://www.androidheadlines.com/2025/10/apple-nears-prompt-ai-acquisition-beating-elon-musks-xai.html)
- Wayve in talks with Microsoft and SoftBank to raise up to \\$2B, FT says - [Biztoc](https://biztoc.com/x/7e7490d4c05ad822)

## Other News

- California enacts child‑protection rules for AI chatbots - [CNET](https://www.cnet.com/videos/apple-tv-plus-drops-the-plus-california-signs-new-ai-regs-into-law-and-amazon-customers-are-upset-about-ads-tech-today/)
- AI mapping reveals hidden ring fault driving Campi Flegrei quakes, logging 54,000 events since 2022 - [Live Science](https://www.livescience.com/planet-earth/volcanos/ai-reveals-hidden-ring-fault-that-is-unleashing-earthquakes-at-italys-campi-flegrei-volcano)
- Microsoft launches MAI‑Image‑1, a faster photorealistic generator coming to Copilot and Bing - [BGR](https://www.bgr.com/1996135/microsoft-ai-image-generator-launch/)
- One in three Americans use AI chatbots for health questions, not for diagnoses - [The Washington Post](https://www.washingtonpost.com/opinions/2025/10/14/ai-chatbots-health-care-diagnoses/)
- Tonal returns to growth, expands retail, adds AI‑powered Reformer Pilates - [Bloomberg](https://www.bloomberg.com/news/articles/2025-10-14/tonal-adds-reformer-pilates-as-it-expands-retail-presence-returns-to-growth)
- DevOps guide to automating Claude Code setup with CLI tricks, MCP servers, container prep, and permissions hardening - [Hacker Noon](https://hackernoon.com/claude-code-setup-the-devops-guide-nobody-warned-you-about)
- DOE worker loses clearance after uploading 187,000 porn images, including some AI‑generated, to a government network - [404 Media](https://www.404media.co/man-stores-ai-generated-robot-porn-on-his-government-computer-loses-access-to-nuclear-secrets/)